In [1]:
import numpy as np

# 8bit对称量化 映射到[-128,127]

https://github.com/InternLM/lmdeploy/blob/main/docs/zh_cn/kv_int8.md

$$
quant = clip(round(\frac {x - zero\_point} {scale}), -127, 128) \\
dequant = quant * scale + zero\_point
$$

In [2]:
np.random.seed(0)
x = np.random.randint(0, 10, (10,)) + np.cos(np.random.uniform(0, 10, (10,)))
x

array([4.01378717, 0.84344637, 2.08489566, 3.06421797, 6.73548049,
       9.08727158, 2.29349957, 4.51461278, 1.0268764 , 4.98035689])

In [3]:
x.min(), x.max()

(0.8434463715450559, 9.087271580440916)

In [4]:
# keep zero_point
zero_point = (x.min() + x.max()) / 2
zero_point

4.965358975992986

In [5]:
# keep scale
scale = (x.max() - x.min()) / 255
scale

0.03232872630939553

In [6]:
# quantize
quant = np.clip(np.round((x - zero_point) / scale), -127, 128)
quant

array([ -29., -127.,  -89.,  -59.,   55.,  127.,  -83.,  -14., -122.,
          0.])

In [7]:
quant.min(), quant.max()

(-127.0, 127.0)

In [8]:
# dequantize
dequant = quant * scale + zero_point
dequant.min(), dequant.max()

(0.8596107346997535, 9.071107217286219)

In [9]:
dequant

array([4.02782591, 0.85961073, 2.08810233, 3.05796412, 6.74343892,
       9.07110722, 2.28207469, 4.51275681, 1.02125437, 4.96535898])

In [10]:
x

array([4.01378717, 0.84344637, 2.08489566, 3.06421797, 6.73548049,
       9.08727158, 2.29349957, 4.51461278, 1.0268764 , 4.98035689])

In [11]:
# 效果看来对称量化比非对称量化效果好
np.isclose(x, dequant, rtol=0.01, atol=0.01)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

# 8bit非对称量化 映射到[0，255]

$$
quant = clip(round(\frac {x - min(x)} {scale}), 0, 255) \\
dequant = quant * scale + min(x)
$$

In [12]:
np.random.seed(0)
x = np.random.randint(0, 10, (10,)) + np.cos(np.random.uniform(0, 10, (10,)))
x

array([4.01378717, 0.84344637, 2.08489566, 3.06421797, 6.73548049,
       9.08727158, 2.29349957, 4.51461278, 1.0268764 , 4.98035689])

In [13]:
# keep min(x)
x_min = x.min()
x_min, x.max()

(0.8434463715450559, 9.087271580440916)

In [14]:
# keep scale
scale = (x.max() - x.min()) / 255
scale

0.03232872630939553

In [22]:
# quantize
quant = np.clip(np.round((x - x_min) / scale), 0, 255)
quant

array([ 98.,   0.,  38.,  69., 182., 255.,  45., 114.,   6., 128.])

In [23]:
quant.min(), quant.max()

(0.0, 255.0)

In [24]:
# dequantize
dequant = quant * scale + x_min
dequant.min(), dequant.max()

(0.8434463715450559, 9.087271580440916)

In [25]:
dequant

array([4.01166155, 0.84344637, 2.07193797, 3.07412849, 6.72727456,
       9.08727158, 2.29823906, 4.52892117, 1.03741873, 4.98152334])

In [26]:
x

array([4.01378717, 0.84344637, 2.08489566, 3.06421797, 6.73548049,
       9.08727158, 2.29349957, 4.51461278, 1.0268764 , 4.98035689])

In [27]:
np.isclose(x, dequant, rtol=0.01, atol=0.01)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])